# Fine-tune UTAE model to predict AGBM from Temporal Sentinel-2 Images

## Imports

In [1]:
from time import time

In [2]:
from accelerate import Accelerator
from biomasstry.datasets import TemporalSentinel2Dataset, TemporalSentinel1Dataset
from biomasstry.models import TemporalSentinelModel, UTAE
from biomasstry.models.unet_tae import ConvBlock
# from biomasstry.models.utils import run_training
import numpy as np
import pandas as pd
from pynvml import *
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
from transformers import TrainingArguments, Trainer, logging
from tqdm.notebook import tqdm

In [3]:
logging.set_verbosity_error()

## Utility Functions

In [4]:
# Utility functions for printing GPU utilization
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [5]:
print_gpu_utilization()

GPU memory occupied: 261 MB.


## Dataset

In [6]:
S3_DIRECT = False  # Access S3 directly or as a mounted data source
if S3_DIRECT:
    data_url="s3://drivendata-competition-biomassters-public-us"
else:
    data_url = ""

datasets = ["Sentinel-1A",  # Sentinel-1 Ascending only
            "Sentinel-1D",  # Sentinel-1 Descending only
            "Sentinel-2all"]

dataset = datasets[2]
if dataset == "Sentinel-1A":
    ds = TemporalSentinel1Dataset(data_url=data_url, bands=["VVA", "VHA"])
    input_nc = 2
    n_tsamples = 6
elif dataset == "Sentinel-1A":
    ds = TemporalSentinel1Dataset(data_url=data_url, bands=["VVD", "VHD"])
    input_nc = 2
    n_tsamples = 6
elif dataset == "Sentinel-2all":
    ds = TemporalSentinel2Dataset(data_url=data_url)
    input_nc = 10
    n_tsamples = 5

In [7]:
torch.manual_seed(0)
train_size = int(0.8*len(ds))
valid_size = len(ds) - train_size
train_set, val_set = random_split(ds, [train_size, valid_size])
print(f"Train samples: {len(train_set)} "
      f"Val. samples: {len(val_set)}")

Train samples: 800 Val. samples: 200


## Model

In [8]:
# model = TemporalSentinelModel(
#     n_tsamples=n_tsamples, 
#     input_nc=input_nc,
#     output_nc=1,
# )  # .to(device)

In [9]:
# HuggingFace Accelerator with Gradient Accumulation
accelerator = Accelerator(gradient_accumulation_steps=4, mixed_precision='fp16')

In [10]:
artifacts_dir = "/notebooks/artifacts"
pretrained_weights_path = artifacts_dir + "/pretrained_utae/f1model.pth.tar"
model = UTAE(input_nc)  # .to(accelerator.device)
saved_dict = torch.load(pretrained_weights_path, map_location=accelerator.device)
model.load_state_dict(saved_dict["state_dict"])
model.out_conv = ConvBlock([32, 32, 1], padding_mode="reflect")

In [11]:
loss_function = nn.MSELoss(reduction='mean')  # .to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training Arguments

In [12]:
default_args = {
    "output_dir": "/notebooks/artifacts",
    "overwrite_output_dir": "True",
    "evaluation_strategy": "steps",
    "num_train_epochs": 10,
    "log_level": "error",
    "report_to": "none",
}

In [13]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    **default_args,
)

## DataLoaders

In [14]:
# DataLoaders
num_workers = 4
train_dataloader = DataLoader(train_set,
                      batch_size=training_args.per_device_train_batch_size,
                      shuffle=True,
                      pin_memory=True,
                      num_workers=num_workers)
eval_dataloader = DataLoader(val_set,
                    batch_size=training_args.per_device_train_batch_size,
                    shuffle=False,
                    pin_memory=True,
                    num_workers=num_workers)

## Prepare

In [15]:
# model = Trainer(model, args=training_args, train_dataset=train_set)

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model,
                                                                optimizer,
                                                                train_dataloader,
                                                                eval_dataloader)

In [16]:
model_name = "UTAE-pretrainedF1"
nb_epochs = 10
date = "20230116"
save_path = artifacts_dir + (f"/{date}_{model_name}_{dataset}_B"
    f"{training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}"
    f"_E{nb_epochs}.pt")
print(f"Model file path: {save_path}")

Model file path: /notebooks/artifacts/20230116_UTAE-pretrainedF1_Sentinel-2all_B16_E10.pt


## Training and Evaluation Loop

In [17]:
num_batches = len(eval_dataloader)
train_metrics = []
val_metrics = []
min_valid_metric = np.inf
# accelerator.save_state(artifacts_dir)
for i in tqdm(range(nb_epochs)):
    train_metrics_epoch = []
    epoch_start = time()
    # model.train()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        inputs = batch["image"]
        targets = batch["target"]
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()
        train_metrics_epoch.append(np.round(np.sqrt(loss.item()), 5))

    epoch_end = time()
    print(f"Epoch training time: {epoch_end - epoch_start}")
    
    # Saving State Dict after each epoch
    torch.save(model.state_dict(), save_path[:-3] + "_Ep{i}.pt")

    # Save after each epoch
    # accelerator.save_state(artifacts_dir)
    
    # Validation Loop
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            inputs = batch["image"]
            targets = batch["target"]
            predictions = model(inputs)
            # Gather all predictions and targets
            all_predictions, all_targets = accelerator.gather_for_metrics((predictions, targets))
            val_loss += loss_function(all_predictions, all_targets).item()

    val_loss /= num_batches
    val_rmse = np.round(np.sqrt(val_loss), 5)
    print(f"Validation Error: \n RMSE: {val_rmse:>8f} \n")
    train_metrics.extend(train_metrics_epoch)
    val_metrics.append((len(train_metrics), val_rmse))
    # check validation score, if improved then save model
    if min_valid_metric > val_rmse:
        print(f'Validation RMSE Decreased({min_valid_metric:.6f}--->{val_rmse:.6f}) \t Saving The Model')
        min_valid_metric = val_rmse

        # Saving State Dict
        torch.save(model.state_dict(), save_path[:-3] + "_BEST.pt")


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch training time: 53.380977392196655


  0%|          | 0/25 [00:00<?, ?it/s]

Validation Error: 
 RMSE:      nan 



...

Epoch training time: 50.018760204315186


  0%|          | 0/25 [00:00<?, ?it/s]

Validation Error: 
 RMSE:      nan 



  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
##### Save the metrics to a file
train_metrics_zipped = list(zip(np.arange(0, len(train_metrics)), train_metrics))
metrics = {'training': train_metrics_zipped, 'validation': val_metrics}
train_metrics_df = pd.DataFrame(metrics['training'], columns=["step", "score"])
val_metrics_df = pd.DataFrame(metrics["validation"], columns=["step", "score"])
train_metrics_df.to_csv(artifacts_dir + "/train_metrics.csv")
val_metrics_df.to_csv(artifacts_dir + "/val_metrics.csv")